In [22]:
# This script identifies pairs of potential duplicates 
# And filters on business name and business status

# We use pandas deduplicate functions to remove exact matches

import json
import pandas as pd
# import re
from rl_helper import strip_accents, AddressClean, haversine
import recordlinkage as rl
from recordlinkage.preprocessing import clean
# from recordlinkage.index import Block
import numpy as np
import math

import mitosheet

In [23]:
# IMPORT DATA

# Read in deduplicate parameters from source json file
# sourcefile=r"ODBiz_Source.json"
# with open(sourcefile) as source_f:
#     Source=json.load(source_f)

# Read in database
df = pd.read_csv("~/ODBiz/6-AssignCSDs/NAICS_Final.csv", low_memory=False, dtype='str')

# TODO we can skip the next step

# Reduce database to only the columns we use for comparisons
# DF=DF[Source["column_map"].values()]
# column_map={val: key for key, val in Source["column_map"].items()}
# DF=DF.rename(columns=column_map)
# df=DF.copy()

In [24]:
# FORMATTTING

# Note/ todo
# i dont want to reformat existing columns. i want to make new ones and delete them at the end

# Remove accents
text_cols=['business_name','full_address','formatted_en','city']
for col in text_cols:
    df.loc[~df[col].isnull(),col]=df.loc[~df[col].isnull(),col].apply(strip_accents)

# Remove periods, apostrophes, commas, and hypens in the Name and address columns
r_list=[r".",r",",r"'",r"-"]

for r in r_list:
    df["business_name"] = df["business_name"].str.replace(r,' ',regex=False)
    df['full_address'] = df['full_address'].str.replace(r,' ',regex=False)

# Remove excess whitespace
df["business_name"] = df["business_name"].str.replace(r" +"," ",regex=True)
df['full_address'] = df['full_address'].str.replace(r" +"," ",regex=True)

# Standardise postal codes - just remove empty space and make sure it's all lower case
df.loc[~df.postal_code.isnull(),'postal_code'] = df.loc[~df.postal_code.isnull(),'postal_code'].str.replace(' ','').str.lower()

# Some records have street number and street name, but no address field filled
df.loc[(df.full_address.isnull())&\
       (~df.formatted_en.isnull()),'full_address']\
    = clean(df.loc[(df.full_address.isnull())&\
       (~df.formatted_en.isnull()),'street_no']+' '+\
           df.loc[(df.full_address.isnull())&\
       (~df.formatted_en.isnull()),'formatted_en']+' '+\
        df.loc[(df.full_address.isnull())&\
       (~df.formatted_en.isnull()),'city'])


In [25]:
# FILTERING

# Remove those with no business name

print('Number removed with business name = NA: ', len(df[df.business_name.isnull()]))
df = df[~df.business_name.isnull()]

# Remove those with inactive status
no_removed = len(df[df.status.isin(['Gone Out of Business', 'Inactive', 'Cancelled'])])

print('Number removed with inactive status: ', no_removed)
df = df[~df.status.isin(['Gone Out of Business', 'Inactive', 'Cancelled'])]

Number removed with business name = NA:  29111
Number removed with inactive status:  58598


In [26]:
df_input = df

In [27]:
# LABEL DUPLICATES (NEW)

# 1. deduplicate and filter strictly on name, street number and street name
df['dupe_1'] = df.duplicated(subset=['business_name', 'street_no', 'formatted_en'], keep=False)

df =  df[df['dupe_1'] == True]


In [28]:
# sort by street number, then street name, then business name
df = df.sort_values(['business_name', 'street_no', 'formatted_en'])


In [29]:
len(df)

112098

In [35]:
# loop and compare duplicates between rows

for i in range(len(df)):
    # check if the street number is the same as the previous row
    if (i > 1):
        if (df.at[df.index[i],'business_name'] == df.at[df.index[i-1],'business_name']):
            # then street name
            if (df.at[df.index[i],'formatted_en'] == df.at[df.index[i-1],'formatted_en']):
                # then name
                if (df.at[df.index[i],'street_no'] == df.at[df.index[i-1],'street_no']):
                    # ok, now we have our potential duplicates
                    # if we match all other variables, then we label duplicate
                    # if 
                    
                    # make a small dataframe with just two rows
                    df2 = df.iloc[[i, i-1]]
                    
                    df2 = df2[['business_name','licence_number', 'postal_code', 'formatted_en', 'business_sector', 'business_description', 'licence_type', 'primary_NAICS']]

                    
                    # remove columns which have NA for either value
                    df2 = df2.dropna(axis=1)
                    
                    
                    # check if the rows are identical
                    if ((df2.iloc[0] == df2.iloc[1]).all()):
                        # duplicate
                        df.at[df.index[i], 'dupe'] = True
                        df.at[df.index[i-1], 'dupe'] = True
                    else:    
                        df.at[df.index[i], 'dupe'] = False
                        # not duplicate
                    
                    
#                     # then province
#                     if (df.at[i, 'province'] == df.at[i-1, 'province']):
#                         # we might also want to check the source is the same
                        
#                         # if they are all the same and we have an x,
#                         # then assign it to the previous row
                        
#                         # check if previous value had an 'x' 
#                         x = df.at[i - 1, 'x']
#                         if (not pd.isna(x) and x != 0):
#                             # if yes, then assign it to the current row
#                             df.at[i, 'x'] = x
                            
#                             # do the same for y
#                             y = df.at[i - 1, 'y']
#                             if (not pd.isna(y) and y != 0):
#                                 df.at[i, 'y'] = y
                    
#                             # also add info other columns from the matching stage
#                             df.at[i, 'matches_r'] = df.at[i - 1, 'matches_r']
#                             df.at[i, 'ratio'] = df.at[i - 1, 'ratio']
#                             df.at[i, 'csdname_oda'] = df.at[i - 1, 'csdname_oda']
#                             df.at[i, 'keep_match'] = df.at[i - 1, 'keep_match']
    

In [36]:
df['dupe'].value_counts()

False    50795
True     20689
Name: dupe, dtype: int64

In [33]:
df['dupe'].isna().sum()

50924

In [37]:
df_dup = df[df['dupe'] == True]

In [38]:
df_dup.to_csv('outputs/dupe.csv', encoding='utf-8')



# mitosheet.sheet(df[(df.dupe_name == False) & (df.dupe_license == False) & (df.dupe_postal == True) & (df.dupe_st_num == True)])


In [15]:
# LABEL DUPLICATES (OLD)

# Rule 1: exact match for business name, license number, street name, postal code - keep one record
df['dupe_1'] = df.duplicated(subset=['business_name', 'street_no', 'formatted_en'], keep=False)
# df['dupe_1'] = df.duplicated(subset=['business_name','licence_number', 'postal_code', 'formatted_en', 'business_sector', 'business_description', 'licence_type', 'primary_NAICS'], keep=False)
print('Rule 1: ', len(df[df['dupe_1'] == True]))
# Results: all duplicate

# Rule 2: matching licence number
# Add - match street name
df['dupe_2'] = df.duplicated(subset=['licence_number', 'formatted_en'], keep=False)
df.loc[df.licence_number.isna(), 'dupe_2'] = False
print('Rule 2: ', len(df[(df.dupe_1 == False) & (df.dupe_2 == True)]))

# Rule 3: business name (within datasets)
df['dupe_3'] = df.duplicated(subset=['business_name', 'street_no', 'formatted_en'], keep=False)
df.loc[df.business_name.isna(), 'dupe_3'] = False
print('Rule 3: ', len(df[(df.dupe_1 == False) & (df.dupe_2 == False) & (df.dupe_3 == True)]))


Rule 1:  112098
Rule 2:  0
Rule 3:  0


In [ ]:
# MERGE AND REMOVE DUPLICATES

# Choose which records to keep and which to remove


In [ ]:
# MERGE DUPLICATES
len(df['du'])



In [40]:
# Label potential duplications
len(df)


# df_dup_1 = df.loc[df.dupe_1==True]
# df_remove = df.duplicated(subset=['business_name','licence_number', 'postal_code', 'street_no'],keep='last')
# df.loc[df.dupe==True].to_csv('outputs/simple_dupes.csv',encoding='utf-8')
# df = df.drop_duplicates(subset=['business_name','full_address','CSDUID', 'licence_number'],keep='last')
# print('No removed with matching name, license, street no and pcode: ', len(df_remove) - len(df))


471255

In [ ]:
# How many are a match, but one record has NA, other has info
# That's the key to what I want to do.

# I want to remove all the duplicates removed in one stage from the successive rounds. 
# I could add a condition to say, when I filter to inspect, if it already has a duplicate 1 mark, leave it.

In [28]:
# Rule 2: exact match for business name, license number, street number, postal code - keep one record
# Business lisence different

df['dupe_2'] = df.duplicated(subset=['business_name', 'postal_code', 'street_no'], keep=False)
print('Additional duplicates found: ', len(df[(df.dupe_1 == False) & (df.dupe_2 == True)]))

# df_dup_2 = df.loc[df.dupe_2==True]
# df_remove = df.duplicated(subset=['business_name', 'postal_code', 'street_no'],keep='last')
# print('number duplicated on these that would be removed: ', len(df_remove))

Number of duplicates found:  63374


In [171]:
# Rule 3: licence number is not and NA and is duplicated



print('Number of duplicates found: ', len(df[df['dupe_3'] == True]))

Number of duplicates found:  13336


In [168]:
len(df[(df.dupe_1 == False) & (df.dupe_2 == False) & (df.dupe_3 == True)])

134499